In [1]:
# Import the required modules
import pandas as pd 
import numpy as np

In [25]:
df_crime = pd.read_csv('/Users/maverick/Downloads/BostonCrime/crimes.csv',  encoding = 'iso-8859-1')
df_offense = pd.read_csv('/Users/maverick/Downloads/BostonCrime/offense_codes.csv',  encoding = 'iso-8859-1')

In [26]:
df_crime.head()

,INCIDENT_NUMBER,OFFENSE_CODE,OFFENSE_CODE_GROUP,OFFENSE_DESCRIPTION,DISTRICT,REPORTING_AREA,SHOOTING,OCCURRED_ON_DATE,YEAR,MONTH,DAY_OF_WEEK,HOUR,UCR_PART,STREET,Lat,Long,Location
0,I182078893,413,Aggravated Assault,ASSAULT - AGGRAVATED - BATTERY,C11,416,Y,9/29/2018 21:12,2018,9,Saturday,21,Part One,WASHINGTON ST,42.281647,-71.071118,"(42.28164735, -71.07111848)"
1,I182078893,2662,Ballistics,BALLISTICS EVIDENCE/FOUND,C11,416,Y,9/29/2018 21:12,2018,9,Saturday,21,Part Two,WASHINGTON ST,42.281647,-71.071118,"(42.28164735, -71.07111848)"
2,I182078872,413,Aggravated Assault,ASSAULT - AGGRAVATED - BATTERY,B2,330,Y,9/29/2018 19:21,2018,9,Saturday,19,Part One,SEAVER ST,42.304554,-71.082954,"(42.30455372, -71.08295366)"
3,I182077958,413,Aggravated Assault,ASSAULT - AGGRAVATED - BATTERY,B3,428,Y,9/26/2018 20:20,2018,9,Wednesday,20,Part One,MORTON ST,42.283746,-71.089382,"(42.28374593, -71.08938232)"
4,I182077958,2662,Ballistics,BALLISTICS EVIDENCE/FOUND,B3,428,Y,9/26/2018 20:20,2018,9,Wednesday,20,Part Two,MORTON ST,42.283746,-71.089382,"(42.28374593, -71.08938232)"


In [27]:
df_offense.head()

,CODE,NAME
0,612,LARCENY PURSE SNATCH - NO FORCE
1,613,LARCENY SHOPLIFTING
2,615,LARCENY THEFT OF MV PARTS & ACCESSORIES
3,1731,INCEST
4,3111,LICENSE PREMISE VIOLATION


In [28]:
df_crime.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110791 entries, 0 to 110790
Data columns (total 17 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   INCIDENT_NUMBER      110791 non-null  object 
 1   OFFENSE_CODE         110791 non-null  int64  
 2   OFFENSE_CODE_GROUP   110791 non-null  object 
 3   OFFENSE_DESCRIPTION  110791 non-null  object 
 4   DISTRICT             110791 non-null  object 
 5   REPORTING_AREA       110791 non-null  object 
 6   SHOOTING             110791 non-null  object 
 7   OCCURRED_ON_DATE     110791 non-null  object 
 8   YEAR                 110791 non-null  int64  
 9   MONTH                110791 non-null  int64  
 10  DAY_OF_WEEK          110791 non-null  object 
 11  HOUR                 110791 non-null  int64  
 12  UCR_PART             110791 non-null  object 
 13  STREET               110791 non-null  object 
 14  Lat                  110791 non-null  float64
 15  Long             

In [29]:
df_offense.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 576 entries, 0 to 575
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   CODE    576 non-null    int64 
 1   NAME    576 non-null    object
dtypes: int64(1), object(1)
memory usage: 9.1+ KB


# q1. 

In [30]:
df_crimeFiltered = df_crime[df_crime['OFFENSE_CODE_GROUP'].isin(['Auto Theft', 'Towed'])]
df1 = df_crimeFiltered[['OCCURRED_ON_DATE', 'MONTH']].copy()

In [31]:
df1.head()

,OCCURRED_ON_DATE,MONTH
80,6/30/2018 0:41,6
278,10/28/2017 1:03,10
345,8/20/2017 0:03,8
373,8/1/2017 19:21,8
424,7/24/2017 16:00,7


In [32]:
df1['OCCURRED_ON_DATE'] = pd.to_datetime(df1['OCCURRED_ON_DATE'], errors='coerce')
if df1['OCCURRED_ON_DATE'].isnull().any():
    print("Warning: There are some NaT values in 'OCCURRED_ON_DATE' after conversion")
# Create phases of the day
df1['PHASE_OF_DAY'] = pd.cut(df1.OCCURRED_ON_DATE.dt.hour,
                             bins=[0, 6, 11, 17, 20, 24],
                             labels=['Night', 'Morning', 'Noon', 'Evening', 'Night'],
                             ordered=False,
                             include_lowest=True)


In [33]:
df1.head()

,OCCURRED_ON_DATE,MONTH,PHASE_OF_DAY
80,2018-06-30 00:41:00,6,Night
278,2017-10-28 01:03:00,10,Night
345,2017-08-20 00:03:00,8,Night
373,2017-08-01 19:21:00,8,Evening
424,2017-07-24 16:00:00,7,Noon


In [34]:
df_pivot = df1.pivot_table(index='PHASE_OF_DAY',
                columns='MONTH', aggfunc='count')
df_pivot.columns = df_pivot.columns.droplevel(0)

In [35]:
df_pivot.head()

MONTH,1,2,3,4,5,6,7,8,9,10,11,12
PHASE_OF_DAY,,,,,,,,,,,,
Evening,51,70,60,70,50,65,82,34,40,32,54,39
Morning,128,175,197,224,315,312,280,223,161,174,170,141
Night,74,140,109,155,145,184,169,98,77,96,109,76
Noon,73,115,119,111,130,139,150,87,86,84,99,66


In [36]:
import calendar
month_dict = {i: calendar.month_name[i] for i in range(1, 13)}
df_pivot.rename(columns=month_dict, inplace=True)
df_pivot.head()

MONTH,January,February,March,April,May,June,July,August,September,October,November,December
PHASE_OF_DAY,,,,,,,,,,,,
Evening,51,70,60,70,50,65,82,34,40,32,54,39
Morning,128,175,197,224,315,312,280,223,161,174,170,141
Night,74,140,109,155,145,184,169,98,77,96,109,76
Noon,73,115,119,111,130,139,150,87,86,84,99,66


# 2 

In [37]:
Crime_Joined = df_crime.merge(df_offense, left_on='OFFENSE_CODE', right_on='CODE', how = 'left')

In [38]:
df2 = Crime_Joined.groupby(['DISTRICT'])['NAME'].apply(lambda x: x.value_counts().index[0]).reset_index()
df2.head()

,DISTRICT,NAME
0,A1,WARRANT ARREST
1,A15,VANDALISM
2,A7,SICK/INJURED/MEDICAL - PERSON
3,B2,VERBAL DISPUTE
4,B3,VERBAL DISPUTE


In [39]:
df_crime.columns

Index(['INCIDENT_NUMBER', 'OFFENSE_CODE', 'OFFENSE_CODE_GROUP',
       'OFFENSE_DESCRIPTION', 'DISTRICT', 'REPORTING_AREA', 'SHOOTING',
       'OCCURRED_ON_DATE', 'YEAR', 'MONTH', 'DAY_OF_WEEK', 'HOUR', 'UCR_PART',
       'STREET', 'Lat', 'Long', 'Location'],
      dtype='object')

In [40]:
# Function to get the offense with maximum occurrence in each district
def get_max_offense_per_district(df):
    return df.groupby('DISTRICT')['OFFENSE_DESCRIPTION'].agg(lambda x: x.mode().iloc[0])

# Get the most common offense per district
max_offense_per_district = get_max_offense_per_district(df_crime)


In [41]:
max_offense_per_district

DISTRICT
A1          ASSAULT SIMPLE - BATTERY
A15                        VANDALISM
A7     SICK/INJURED/MEDICAL - PERSON
B2                    VERBAL DISPUTE
B3                    VERBAL DISPUTE
C11               INVESTIGATE PERSON
C6     SICK/INJURED/MEDICAL - PERSON
D14              TOWED MOTOR VEHICLE
D4               LARCENY SHOPLIFTING
E13    SICK/INJURED/MEDICAL - PERSON
E18    SICK/INJURED/MEDICAL - PERSON
E5     SICK/INJURED/MEDICAL - PERSON
Name: OFFENSE_DESCRIPTION, dtype: object

# 3

In [21]:
df_crime.sort_values(['DISTRICT','OCCURRED_ON_DATE'], ascending=[True, True], inplace=True)
df_crime['DATE_OF_PREVIOUS_INCIDENT'] = df_crime.groupby(['DISTRICT'])['OCCURRED_ON_DATE'].shift(1)
df_crime.head()

,INCIDENT_NUMBER,OFFENSE_CODE,OFFENSE_CODE_GROUP,OFFENSE_DESCRIPTION,DISTRICT,REPORTING_AREA,SHOOTING,OCCURRED_ON_DATE,YEAR,MONTH,DAY_OF_WEEK,HOUR,UCR_PART,STREET,Lat,Long,Location,PRE_OFF_DATE,DATE_OF_PREVIOUS_INCIDENT
85738,I162046288,3115,Investigate Person,INVESTIGATE PERSON,A1,90,N,1/1/2016 0:00,2016,1,Friday,0,Part Three,COMMERCIAL ST,42.365791,-71.051162,"(42.36579076, -71.05116185)",NaN,NaN
106169,I162000328,3201,Property Lost,PROPERTY - LOST,A1,122,N,1/1/2016 0:15,2016,1,Friday,0,Part Three,TREMONT ST,42.349429,-71.065366,"(42.34942864, -71.06536641)",1/1/2016 0:00,1/1/2016 0:00
106170,I162000328,3111,License Violation,LICENSE PREMISE VIOLATION,A1,122,N,1/1/2016 0:15,2016,1,Friday,0,Part Three,TREMONT ST,42.349429,-71.065366,"(42.34942864, -71.06536641)",1/1/2016 0:15,1/1/2016 0:15
106350,I162000117,612,Larceny,LARCENY PURSE SNATCH - NO FORCE,A1,102,N,1/1/2016 0:16,2016,1,Friday,0,Part One,TREMONT ST,42.356585,-71.061895,"(42.35658500, -71.06189505)",1/1/2016 0:15,1/1/2016 0:15
106188,I162000306,1106,Confidence Games,FRAUD - CREDIT CARD / ATM FRAUD,A1,127,N,1/1/2016 0:58,2016,1,Friday,0,Part Two,WASHINGTON ST,42.348761,-71.064083,"(42.34876087, -71.06408261)",1/1/2016 0:16,1/1/2016 0:16


# 4

In [44]:
def get_max_incident_street_per_district(df):
    return df.groupby(['DISTRICT', 'STREET']).size().reset_index(name='count').sort_values('count', ascending=False).drop_duplicates('DISTRICT').set_index('DISTRICT')['STREET']

# street with the maximum number of incidents per district
max_incident_street_per_district = get_max_incident_street_per_district(df_crime)


In [43]:
max_incident_street_per_district

DISTRICT
D4           BOYLSTON ST
B3         BLUE HILL AVE
C11       DORCHESTER AVE
E18        HYDE PARK AVE
D14     COMMONWEALTH AVE
A1         WASHINGTON ST
B2             DUDLEY ST
E13            CENTRE ST
C6     MASSACHUSETTS AVE
E5         WASHINGTON ST
A7             BORDER ST
A15       BUNKER HILL ST
Name: STREET, dtype: object

# 5 

In [46]:
# Create a subset of data with only the 10 most recent incidents for each street
df_crime['OCCURRED_ON_DATE'] = pd.to_datetime(df_crime['OCCURRED_ON_DATE'])
recent_10_incidents_per_street = df_crime.sort_values('OCCURRED_ON_DATE', ascending=False).groupby('STREET').head(10)
print("\nSubset of data with only the 10 most recent incidents for each street:")
recent_10_incidents_per_street.head()


Subset of data with only the 10 most recent incidents for each street:


,INCIDENT_NUMBER,OFFENSE_CODE,OFFENSE_CODE_GROUP,OFFENSE_DESCRIPTION,DISTRICT,REPORTING_AREA,SHOOTING,OCCURRED_ON_DATE,YEAR,MONTH,DAY_OF_WEEK,HOUR,UCR_PART,STREET,Lat,Long,Location
0,I182078893,413,Aggravated Assault,ASSAULT - AGGRAVATED - BATTERY,C11,416,Y,2018-09-29 21:12:00,2018,9,Saturday,21,Part One,WASHINGTON ST,42.281647,-71.071118,"(42.28164735, -71.07111848)"
1,I182078893,2662,Ballistics,BALLISTICS EVIDENCE/FOUND,C11,416,Y,2018-09-29 21:12:00,2018,9,Saturday,21,Part Two,WASHINGTON ST,42.281647,-71.071118,"(42.28164735, -71.07111848)"
2,I182078872,413,Aggravated Assault,ASSAULT - AGGRAVATED - BATTERY,B2,330,Y,2018-09-29 19:21:00,2018,9,Saturday,19,Part One,SEAVER ST,42.304554,-71.082954,"(42.30455372, -71.08295366)"
3,I182077958,413,Aggravated Assault,ASSAULT - AGGRAVATED - BATTERY,B3,428,Y,2018-09-26 20:20:00,2018,9,Wednesday,20,Part One,MORTON ST,42.283746,-71.089382,"(42.28374593, -71.08938232)"
4,I182077958,2662,Ballistics,BALLISTICS EVIDENCE/FOUND,B3,428,Y,2018-09-26 20:20:00,2018,9,Wednesday,20,Part Two,MORTON ST,42.283746,-71.089382,"(42.28374593, -71.08938232)"
